In [1]:
from __future__ import print_function
import torch
import argparse
import sys
from torch.utils.data import DataLoader,random_split
from torchvision import datasets, models, transforms
import platform
from datetime import timedelta, date, datetime
from model.vdsr import vdsr
import torch.nn as nn
import numpy as np
import torch.optim as optim
import os
from math import log10
import time
from util.PrepareData import ACCESS_BARRA_v2_0,ACCESS_BARRA_v2_1,ACCESS_BARRA_vdsr



def write_log(log,args):
    print(log)
    if not os.path.exists("./save/"+args.train_name+"/"):
        os.mkdir("./save/"+args.train_name+"/")
    my_log_file=open("./save/"+args.train_name + '/train.txt', 'a')
#     log="Train for batch %d,data loading time cost %f s"%(batch,start-time.time())
    my_log_file.write(log + '\n')
    my_log_file.close()
    return


parser = argparse.ArgumentParser(description='PyTorch Super Res Example')
# Hardware specifications
parser.add_argument('--n_threads', type=int, default=0,
                    help='number of threads for data loading')

parser.add_argument('--cpu', action='store_true',help='cpu only?') 
parser.add_argument('--test', action='store_true',help='cpu only?') 

# hyper-parameters
parser.add_argument('--train_name', type=str, default="vdsr_pr", help='training name')

parser.add_argument('--batch_size', type=int, default=4, help='training batch size')
parser.add_argument('--testBatchSize', type=int, default=4, help='testing batch size')
parser.add_argument('--nEpochs', type=int, default=200, help='number of epochs to train for')
parser.add_argument('--lr', type=float, default=0.01, help='Learning Rate. Default=0.01')
parser.add_argument('--seed', type=int, default=123, help='random seed to use. Default=123')

# model configuration
parser.add_argument('--upscale_factor', '-uf',  type=int, default=4, help="super resolution upscale factor")
parser.add_argument('--model', '-m', type=str, default='vdsr', help='choose which model is going to use')

#data
parser.add_argument('--pr', type=bool, default=True,help='add-on pr?')

parser.add_argument('--train_start_time', type=type(datetime(1990,1,25)), default=datetime(1990,1,2),help='r?')
parser.add_argument('--train_end_time', type=bool, default=datetime(1990,2,9),help='?')
parser.add_argument('--test_start_time', type=bool, default=datetime(2012,1,1),help='a?')
parser.add_argument('--test_end_time', type=bool, default=datetime(2012,12,24),help='')

parser.add_argument('--start_date', type=bool, default=datetime(1990,1,2),help='a?')
parser.add_argument('--end_date', type=bool, default=datetime(2012,12,24),help='')


parser.add_argument('--dem', action='store_true',help='add-on dem?') 
parser.add_argument('--psl', action='store_true',help='add-on psl?') 
parser.add_argument('--zg', action='store_true',help='add-on zg?') 
parser.add_argument('--tasmax', action='store_true',help='add-on tasmax?') 
parser.add_argument('--tasmin', action='store_true',help='add-on tasmin?')
parser.add_argument('--leading_time_we_use', type=int,default=1,help='add-on tasmin?')
parser.add_argument('--ensemble', type=int, default=11,help='total ensambles is 11') 
parser.add_argument('--channels', type=float, default=0,help='channel of data_input must') 
#[111.85, 155.875, -44.35, -9.975]
parser.add_argument('--domain', type=list, default=[112.9, 154.25, -43.7425, -9.0],help='dataset directory')

parser.add_argument('--file_ACCESS_dir', type=str, default="../data/",help='dataset directory')
parser.add_argument('--file_BARRA_dir', type=str, default="../data/barra_aus/",help='dataset directory')
parser.add_argument('--file_DEM_dir', type=str, default="../DEM/",help='dataset directory')
parser.add_argument('--precision', type=str, default='single',choices=('single', 'half','double'),help='FP precision for test (single | half)')

args = parser.parse_args([])




E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
Data=torch.rand((11,1,150,150))
net=vdsr()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if torch.cuda.device_count() > 1:
    write_log("!!!!!!!!!!!!!Let's use"+str(torch.cuda.device_count())+"GPUs!",args)
    net = nn.DataParallel(net,range(torch.cuda.device_count()))
else:
    write_log("Let's use"+str(torch.cuda.device_count())+"GPUs!",args)

net.to(device)
Data=Data.to(device)

Let's use1GPUs!


In [7]:
ens=torch.rand((11,3))
obs=torch.rand(316,376)
output=net(ens)

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [64, 1, 3, 3], but got 3-dimensional input of size [316, 376, 11] instead

In [5]:
optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=0.9, weight_decay=1e-4)

for e in range(args.nEpochs):
    loss=0
    start=time.time()
    ens=torch.rand((316,376,11))
    obs=torch.rand(316,376)
    
    optimizer.zero_grad()
    with torch.set_grad_enabled(True):
        output=net(Data)
        running_loss =criterion(ens, obs)
        running_loss.backward()
        optimizer.step()
    loss+=running_loss.item()

In [6]:
import climpred


ModuleNotFoundError: No module named 'climpred'

In [ ]:
pip install